**Goal:** Check if the extraction and transformation of data is correct.

# Initialize

In [4]:
from utils import read_config, DfInfo
from extract import etl_extract
from transform import etl_transform
from IPython.display import display

In [2]:
# Input values
config_file = 'dl.cfg'
src = 'local'

# Read input
config = read_config(config_file)
base_path = config[src]['base_path']
song_path = config[src]['song_path']
log_path = config[src]['log_path']
output_path = config[src]['output_path']
aws_creds = config['AWS']

# Read JSON files (EXTRACT)
song_log_data = etl_extract(base_path, song_path, log_path, aws_creds)
song_data = song_log_data['song_data']
log_data = song_log_data['log_data']

# Process data (TRANSFORM)
tables = etl_transform(song_data, log_data)

[2020-05-18 00:08:17,987] INFO - Running `etl_extract`...
[2020-05-18 00:08:17,988] INFO - Running `create_spark_session`...
[2020-05-18 00:08:20,900] INFO - `create_spark_session` finished!
[2020-05-18 00:08:20,901] INFO - Running `read_json_data`...
[2020-05-18 00:08:22,872] INFO - `read_json_data` finished!
[2020-05-18 00:08:22,873] INFO - Running `read_json_data`...
[2020-05-18 00:08:23,057] INFO - `read_json_data` finished!
[2020-05-18 00:08:23,059] INFO - `etl_extract` finished!
[2020-05-18 00:08:23,060] INFO - Running `etl_transform`...
[2020-05-18 00:08:23,724] INFO - Running `create_songs_table`...
[2020-05-18 00:08:23,823] INFO - `create_songs_table` finished!
[2020-05-18 00:08:23,823] INFO - Running `create_artists_table`...
[2020-05-18 00:08:23,879] INFO - `create_artists_table` finished!
[2020-05-18 00:08:23,879] INFO - Running `create_users_table`...
[2020-05-18 00:08:24,267] INFO - `create_users_table` finished!
[2020-05-18 00:08:24,268] INFO - Running `create_time_table

In [5]:
tables_full = dict({'song_data': DfInfo('song_data', song_data, None, None),
                    'log_data': DfInfo('log_data', log_data, None, None)},
                   **tables)

# Count rows

In [6]:
print('Number of rows')
print('--------------')
width = max(len(t.name) for t in tables_full.values())
for t in tables_full.values():
    print('{table:{width:d}s} = {rows:,d}'
          ''.format(table=t.name,
                    width=width,
                    rows=t.df.count()))

print()

log_data_songplays = tables['log_data_songplays'].df

print('Number of distinct rows in the original data')
print('(must match the values above)')
print('--------------------------------------------')
print('song ID   = {:,d}'.format(song_data.select('song_id').distinct().count()))
print('artist ID = {:,d}'.format(song_data.select('artist_id').distinct().count()))
print('user ID   = {:,d} (all), {:,d} (songplays)'
      ''.format(log_data.select('userId').distinct().count(),
                log_data_songplays.select('userId').distinct().count()))
print('timestamp = {:,d} (all), {:,d} (songplays)'
      ''.format(log_data.select('ts').distinct().count(),
                log_data_songplays.select('ts').distinct().count()))

Number of rows
--------------
song_data          = 71
log_data           = 8,056
log_data_songplays = 6,820
songs              = 71
artists            = 69
users              = 96
time               = 6,813
songplays          = 1

Number of distinct rows in the original data
(must match the values above)
--------------------------------------------
song ID   = 71
artist ID = 69
user ID   = 98 (all), 96 (songplays)
timestamp = 8,023 (all), 6,813 (songplays)


# First few rows of all tables and schemas

In [7]:
for t in tables_full.values():
    print(80 * '-')
    print(t.name)
    display(t.df.limit(10).toPandas())
    t.df.printSchema()

--------------------------------------------------------------------------------
song_data


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARDR4AC1187FB371A1,NaN,,NaN,Montserrat Caballé;Placido Domingo;Vicente Sar...,511.16363,1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,0
1,AREBBGV1187FB523D2,NaN,"Houston, TX",NaN,Mike Jones (Featuring CJ_ Mello & Lil' Bran),173.66159,1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),0
2,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
3,ARPBNLO1187FB3D52F,40.71455,"New York, NY",-74.00712,Tiny Tim,43.36281,1,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,2000
4,ARDNS031187B9924F0,32.67828,Georgia,-83.22295,Tim Wilson,186.48771,1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,2005
5,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
6,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0
7,ARPFHN61187FB575F6,41.88415,"Chicago, IL",-87.63241,Lupe Fiasco,279.97995,1,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),0
8,ARI2JSK1187FB496EF,51.50632,"London, England",-0.12714,Nick Ingman;Gavyn Wright,111.62077,1,SODUJBS12A8C132150,Wessex Loses a Bride,0
9,AR0RCMP1187FB3F427,30.08615,"Beaumont, TX",-94.10158,Billie Jo Spears,133.32853,1,SOGXHEG12AB018653E,It Makes No Difference Now,1992


root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: integer (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)

--------------------------------------------------------------------------------
log_data


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12
5,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
6,None,Logged In,Samuel,M,1,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,About,1.540493e+12,597,None,200,1542253460796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
7,None,Logged Out,None,None,0,None,NaN,paid,None,PUT,Login,NaN,602,None,307,1542260074796,None,
8,None,Logged In,Tegan,F,1,Levine,NaN,paid,"Portland-South Portland, ME",GET,Home,1.540794e+12,602,None,200,1542260277796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
9,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

--------------------------------------------------------------------------------
log_data_songplays


...

root
 |-- user_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

--------------------------------------------------------------------------------
time


,start_time,hour,day,week,month,year,weekday
0,2018-11-21 08:13:42,8,21,47,11,2018,4
1,2018-11-21 09:01:05,9,21,47,11,2018,4
2,2018-11-21 16:34:01,16,21,47,11,2018,4
3,2018-11-21 18:44:26,18,21,47,11,2018,4
4,2018-11-14 01:37:52,1,14,46,11,2018,4
5,2018-11-14 02:13:46,2,14,46,11,2018,4
6,2018-11-14 15:13:34,15,14,46,11,2018,4
7,2018-11-28 14:10:26,14,28,48,11,2018,4
8,2018-11-05 02:40:25,2,5,45,11,2018,2
9,2018-11-05 12:39:43,12,5,45,11,2018,2


root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

--------------------------------------------------------------------------------
songplays


,songplay_id,start_time,year,month,user_id,level,song_id,artist_id,session_id,location,user_agent
0,0,2018-11-21 19:56:47,2018,11,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)

